In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import Input, LocallyConnected1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 20
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 20
N_FEATURES = 10

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 1000

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/'

In [15]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],20,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [16]:
n_length =  20
n_depth=10
n_channel=10
n_outputs=52

In [17]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,10)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model   

In [18]:
if __name__ == "__main__":
    model = generate_model()

(None, 20, 10)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
(None, 20, 10, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 12800)
(None, 52)
inputs_shape (None, 20, 10)


In [19]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
#     feature standardization
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_length =  20
    n_depth=10
    n_channel=10
    x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5'
#     model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    test_mean=statistics.mean(test_acc)
    print('test_mean for %d subjects:',i)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (9099, 20, 10)
9099 training samples
y_train shape:  (9099,)
num_time_periods 20
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (20, 10)
input_shape: (20, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (9099, 52)
x

10/10 [==============================] - ETA: 0s - loss: 5.8801 - accuracy: 0.0635
Epoch 00001: val_accuracy improved from -inf to 0.09797, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 16s 2s/step - loss: 5.8801 - accuracy: 0.0635 - val_loss: 5.7292 - val_accuracy: 0.0980 - lr: 0.0010
Epoch 2/120
 9/10 [==========================>...] - ETA: 0s - loss: 5.2919 - accuracy: 0.1527
Epoch 00002: val_accuracy improved from 0.09797 to 0.20800, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 106ms/step - loss: 5.2883 - accuracy: 0.1533 - val_loss: 5.3295 - val_accuracy: 0.2080 - lr: 0.0010
Epoch 3/120
 9/10 [==========================>...] - ETA: 0s - loss: 4.7886 - accuracy: 0.2374
Epoch 00003: val_accuracy improved from 0.20800 to 0.32060, saving model to /media/naveen/nav/mat_codes/nina_DB1

 9/10 [==========================>...] - ETA: 0s - loss: 1.7699 - accuracy: 0.6877
Epoch 00022: val_accuracy did not improve from 0.70659
10/10 [==============================] - 1s 81ms/step - loss: 1.7712 - accuracy: 0.6868 - val_loss: 1.6949 - val_accuracy: 0.6976 - lr: 0.0010
Epoch 23/120
10/10 [==============================] - ETA: 0s - loss: 1.7306 - accuracy: 0.6873
Epoch 00023: val_accuracy did not improve from 0.70659
10/10 [==============================] - 1s 76ms/step - loss: 1.7306 - accuracy: 0.6873 - val_loss: 1.7352 - val_accuracy: 0.6863 - lr: 0.0010
Epoch 24/120
10/10 [==============================] - ETA: 0s - loss: 1.6891 - accuracy: 0.6950
Epoch 00024: val_accuracy improved from 0.70659 to 0.71711, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 122ms/step - loss: 1.6891 - accuracy: 0.6950 - val_loss: 1.6156 - val_accuracy: 0.7171 - lr: 0.0010
Epoch 25/120
 9/10 

 9/10 [==========================>...] - ETA: 0s - loss: 1.1830 - accuracy: 0.7963
Epoch 00045: val_accuracy improved from 0.76917 to 0.76968, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 122ms/step - loss: 1.1846 - accuracy: 0.7957 - val_loss: 1.2658 - val_accuracy: 0.7697 - lr: 1.0000e-04
Epoch 46/120
10/10 [==============================] - ETA: 0s - loss: 1.1803 - accuracy: 0.7937
Epoch 00046: val_accuracy did not improve from 0.76968
10/10 [==============================] - 1s 86ms/step - loss: 1.1803 - accuracy: 0.7937 - val_loss: 1.2656 - val_accuracy: 0.7671 - lr: 1.0000e-04
Epoch 47/120
10/10 [==============================] - ETA: 0s - loss: 1.1734 - accuracy: 0.7996
Epoch 00047: val_accuracy improved from 0.76968 to 0.77097, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s

Epoch 70/120
10/10 [==============================] - ETA: 0s - loss: 1.0672 - accuracy: 0.8225
Epoch 00070: val_accuracy did not improve from 0.78123
10/10 [==============================] - 1s 71ms/step - loss: 1.0672 - accuracy: 0.8225 - val_loss: 1.1851 - val_accuracy: 0.7797 - lr: 1.0000e-04
Epoch 71/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.0470 - accuracy: 0.8310
Epoch 00071: val_accuracy did not improve from 0.78123
10/10 [==============================] - 1s 69ms/step - loss: 1.0478 - accuracy: 0.8310 - val_loss: 1.1861 - val_accuracy: 0.7779 - lr: 1.0000e-04
Epoch 72/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.0560 - accuracy: 0.8273
Epoch 00072: val_accuracy did not improve from 0.78123
10/10 [==============================] - 1s 75ms/step - loss: 1.0580 - accuracy: 0.8272 - val_loss: 1.1880 - val_accuracy: 0.7776 - lr: 1.0000e-04
Epoch 73/120
10/10 [==============================] - ETA: 0s - loss: 1.0544 - accuracy: 0.8257
Epoch 0007

...

 9/10 [==========================>...] - ETA: 0s - loss: 4.0828 - accuracy: 0.1346
Epoch 00001: val_accuracy improved from -inf to 0.23750, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 3s 345ms/step - loss: 4.0747 - accuracy: 0.1355 - val_loss: 3.3163 - val_accuracy: 0.2375 - lr: 0.0010
Epoch 2/120
 9/10 [==========================>...] - ETA: 0s - loss: 3.0639 - accuracy: 0.2398
Epoch 00002: val_accuracy improved from 0.23750 to 0.30546, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 97ms/step - loss: 3.0617 - accuracy: 0.2400 - val_loss: 2.7581 - val_accuracy: 0.3055 - lr: 0.0010
Epoch 3/120
 9/10 [==========================>...] - ETA: 0s - loss: 2.7192 - accuracy: 0.3068
Epoch 00003: val_accuracy improved from 0.30546 to 0.37728, saving model to /media/naveen/nav/mat_codes/nina_DB

Epoch 23/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.3565 - accuracy: 0.6476
Epoch 00023: val_accuracy did not improve from 0.64401
10/10 [==============================] - 1s 64ms/step - loss: 1.3577 - accuracy: 0.6473 - val_loss: 1.4120 - val_accuracy: 0.6414 - lr: 0.0010
Epoch 24/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.3631 - accuracy: 0.6448
Epoch 00024: val_accuracy improved from 0.64401 to 0.64837, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 97ms/step - loss: 1.3649 - accuracy: 0.6449 - val_loss: 1.3893 - val_accuracy: 0.6484 - lr: 0.0010
Epoch 25/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.3203 - accuracy: 0.6589
Epoch 00025: val_accuracy did not improve from 0.64837
10/10 [==============================] - 1s 64ms/step - loss: 1.3205 - accuracy: 0.6582 - val_loss: 1.4913 - val_accuracy: 0.6325 - lr: 0.0010
Epoch 2

Epoch 47/120
10/10 [==============================] - ETA: 0s - loss: 1.0482 - accuracy: 0.7285
Epoch 00047: val_accuracy did not improve from 0.69505
10/10 [==============================] - 1s 70ms/step - loss: 1.0482 - accuracy: 0.7285 - val_loss: 1.2785 - val_accuracy: 0.6848 - lr: 1.0000e-04
Epoch 48/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.0415 - accuracy: 0.7292
Epoch 00048: val_accuracy did not improve from 0.69505
10/10 [==============================] - 1s 66ms/step - loss: 1.0463 - accuracy: 0.7278 - val_loss: 1.2916 - val_accuracy: 0.6809 - lr: 1.0000e-04
Epoch 49/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.0222 - accuracy: 0.7442
Epoch 00049: val_accuracy did not improve from 0.69505
10/10 [==============================] - 1s 65ms/step - loss: 1.0257 - accuracy: 0.7432 - val_loss: 1.2609 - val_accuracy: 0.6879 - lr: 1.0000e-04
Epoch 50/120
 9/10 [==========================>...] - ETA: 0s - loss: 1.0291 - accuracy: 0.7302
Epoch 0005

 9/10 [==========================>...] - ETA: 0s - loss: 0.9741 - accuracy: 0.7500
Epoch 00072: val_accuracy did not improve from 0.70197
10/10 [==============================] - 1s 66ms/step - loss: 0.9753 - accuracy: 0.7496 - val_loss: 1.2268 - val_accuracy: 0.6953 - lr: 1.0000e-04
Epoch 73/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9750 - accuracy: 0.7479
Epoch 00073: val_accuracy improved from 0.70197 to 0.70223, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_sub_20X10/checkpoint.hdf5
10/10 [==============================] - 1s 89ms/step - loss: 0.9773 - accuracy: 0.7470 - val_loss: 1.2214 - val_accuracy: 0.7022 - lr: 1.0000e-04
Epoch 74/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9678 - accuracy: 0.7551
Epoch 00074: val_accuracy did not improve from 0.70223
10/10 [==============================] - 1s 64ms/step - loss: 0.9698 - accuracy: 0.7548 - val_loss: 1.2326 - val_accuracy: 0.6948 - lr: 1.0000e-04
Epoch 75

Epoch 99/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9369 - accuracy: 0.7623
Epoch 00099: val_accuracy did not improve from 0.70813
10/10 [==============================] - 1s 65ms/step - loss: 0.9383 - accuracy: 0.7621 - val_loss: 1.2189 - val_accuracy: 0.6976 - lr: 1.0000e-05
Epoch 100/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9322 - accuracy: 0.7616
Epoch 00100: val_accuracy did not improve from 0.70813
10/10 [==============================] - 1s 66ms/step - loss: 0.9315 - accuracy: 0.7617 - val_loss: 1.2098 - val_accuracy: 0.7009 - lr: 1.0000e-05
Epoch 101/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9322 - accuracy: 0.7629
Epoch 00101: val_accuracy did not improve from 0.70813
10/10 [==============================] - 1s 67ms/step - loss: 0.9348 - accuracy: 0.7622 - val_loss: 1.2048 - val_accuracy: 0.7071 - lr: 1.0000e-05
Epoch 102/120
 9/10 [==========================>...] - ETA: 0s - loss: 0.9419 - accuracy: 0.7584
Epoch 0

In [20]:
statistics.mean(test_acc)

0.7425740648199011